# Lecture 8: Approximation Methods and Root Finding

## Part 1: Interpolation
-------------

Given a set of *N* points $(x_i, y_i)$ with $i = 1, 2, …N$, we sometimes need a function $\hat{f}(x)$ which returns $y_i = f(x_i)$ where $x == x_i$, and which in addition provides some interpolation of the data $(x_i, y_i)$ for all $x$.

The function `y0 = scipy.interpolate.interp1d(x,y,kind=’..’)` does this interpolation based on splines of varying order. Note that the function `interp1d` returns *a function* `y0` which will then interpolate the x-y data for any given $x$ when called as $y0(x)$.

The code below demonstrates this, and shows the different interpolation kinds.

In [ ]:
import numpy as np
import scipy.interpolate
import pylab
%matplotlib inline

def create_data(n):
    """Given an integer n, returns n data points
    x and values y as a numpy.array."""
    xmax = 5.
    x = np.linspace(0, xmax, n)
    y = - x**2
    #make y-data somewhat irregular
    y += 1.5 * np.random.normal(size=len(x))
    return x, y

#main program
n = 10
x, y = create_data(n)

#use finer and regular mesh for plot
xfine = np.linspace(0.1, 4.9, n * 100)
#interpolate with piecewise constant function (p=0)
y0 = scipy.interpolate.interp1d(x, y, kind='nearest')
#interpolate with piecewise linear func (p=1)
y1 = scipy.interpolate.interp1d(x, y, kind='linear')
#interpolate with piecewise cubic func (p=3)
y3 = scipy.interpolate.interp1d(x, y, kind='cubic')
print(y3(4.5))
#interpolate with cubic Hermite
yfineH = scipy.interpolate.pchip_interpolate(x, y, xfine)

pylab.figure()
pylab.plot(x, y, 'o', label='data point')
pylab.plot(xfine, y0(xfine), label='nearest')
pylab.plot(xfine, y1(xfine), label='linear')
pylab.plot(xfine, y3(xfine), label='cubic')
pylab.plot(xfine, yfineH, label='cubic Hermite')
pylab.legend(loc='upper left',bbox_to_anchor=(1.02,0.5,0.5,0.5))
pylab.xlabel('x')
pylab.ylabel('y')

# plot differences
pylab.figure()
pylab.plot(x, y+x**2, 'o', label='data point')
pylab.plot(xfine, y0(xfine)+xfine**2, label='nearest')
pylab.plot(xfine, y1(xfine)+xfine**2, label='linear')
pylab.plot(xfine, y3(xfine)+xfine**2, label='cubic')
pylab.plot(xfine, yfineH+xfine**2, label='cubic Hermite')
pylab.legend(loc='upper left',bbox_to_anchor=(1.02,0.5,0.5,0.5))
pylab.xlabel('x')
pylab.ylabel('y-y_function = y+x**2')


## Part 2: Root finding

### Root finding using iterative relaxation method

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

def g(x):
    return  np.sin(x) # 2-np.exp(-x) #   np.cos(x) # 

nIter = 50

na = range(nIter)
xa = np.zeros(nIter)
x = 0.2 # initial guess
for k in na:
    xa[k] = x
    x = g(x)
    
plt.plot(na,xa)
plt.xlabel('n')
plt.ylabel('x')

print('Solution = ',xa[-1],', precision = ',np.abs(xa[-1]-xa[-2]))

### Root finding using the bisection method

First we introduce the `bisect` algorithm which is (i) robust and (ii) slow but conceptually very simple.

Suppose we need to compute the roots of *f*(*x*)=*x*<sup>3</sup> − 2*x*<sup>2</sup>. This function has a (double) root at *x* = 0 (this is trivial to see) and another root which is located between *x* = 1.5 (where *f*(1.5)= − 1.125) and *x* = 3 (where *f*(3)=9). It is pretty straightforward to see that this other root is located at *x* = 2. Here is a program that determines this root numerically:

In [ ]:
from scipy.optimize import bisect

def f(x):
    """returns f(x)=x^3-2x^2. Has roots at
    x=0 (double root) and x=2"""
    return x ** 3 - 2 * x ** 2

# main program starts here
x = bisect(f, 1.5, 3, xtol=1e-6)

print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x))
print("The exact error is %g." % (2 - x))

The `bisect()` method takes three compulsory arguments: (i) the function *f*(*x*), (ii) a lower limit *a* (for which we have chosen 1.5 in our example) and (ii) an upper limit *b* (for which we have chosen 3). The optional parameter `xtol` determines the maximum error of the method.

### Root finding using Müller-Brent method

This is a classic method to find a zero of the function f on the sign changing interval [a , b]. It is a safe version of the secant method that uses inverse quadratic extrapolation. Brent’s method combines root bracketing, interval bisection, and inverse quadratic interpolation.

In [ ]:
from scipy.optimize import brentq

def f(x):
    return x ** 3 - 2 * x ** 2

x = brentq(f, 1.5, 3, xtol=1e-6)

print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x))
print("The exact error is %g." % (2 - x))

### Root finding using the `fsolve` function

A (often) better (in the sense of “more efficient”) algorithm than the bisection algorithm is implemented in the general purpose `fsolve()` function for root finding of (multidimensional) functions. This algorithm needs only one starting point close to the suspected location of the root (but is not guaranteed to converge).

Here is an example:

In [ ]:
from scipy.optimize import fsolve

def f(x):
    return x ** 3 - 2 * x ** 2

x0 = [-0.5,2.5]
x = fsolve(f, x0 )           # specify starting points

print("Number of roots is",len(x))
print("The root(s) are ",x)
print("Error of the initial guess = ",x0[0]-0,x0[1]-2)
print("error : ",x[0]-0,x[1]-2)

The input to `fsolve` is the function and the array of initial locations for the roots. The return value of `fsolve` is a numpy array of length *n* for a root finding problem with *n* variables. In the example above, we have *n* = 2.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

x1=np.linspace(-1,3,100)
fig, ax = plt.subplots()
ax.plot(x1, f(x1))
ax.set(xlabel='x', ylabel='f(x)',
       title='Visual check')
plt.axhline(y=0, c="black")
plt.axvline(x=0, c="black")
ax.grid()
plt.show()